## To export to tf_lite we cant do it directly, so what we do is to export to ONNX and then to tf_lite

In [ ]:
from ultralytics import YOLO

model = YOLO('./best.pt')
model.export(format='tflite')

In [7]:
from ultralytics import YOLO
import onnx2tf

model = YOLO('./best.pt')
model.export(format='onnx',opset=15)

Ultralytics YOLOv8.2.19  Python-3.10.4 torch-2.3.0+cu118 CPU (11th Gen Intel Core(TM) i7-11370H 3.30GHz)
YOLOv9c summary (fused): 384 layers, 25326187 parameters, 0 gradients, 102.4 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 288, 288) BCHW and output shape(s) (1, 13, 1701) (49.2 MB)

ONNX: starting export with onnx 1.16.1 opset 15...
ONNX: export success  2.9s, saved as 'best.onnx' (96.8 MB)

Export complete (5.6s)
Results saved to D:\projects\freelance\saudi cairo uni student\work part 2\98 - Copy
Predict:         yolo predict task=detect model=best.onnx imgsz=288  
Validate:        yolo val task=detect model=best.onnx imgsz=288 data=/kaggle/working/Tomata-Leaf-Disease-3/data.yaml  
Visualize:       https://netron.app


'best.onnx'

# from ONNX to TF 

In [10]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load("./best.onnx")

# Convert ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model to a temporary directory
tf_rep.export_graph("./temp")

# default settings without optimizations

In [19]:
import tensorflow as tf

tf_model_path_directory = './temp/'
tflite_model_path = './tf_lite_model.tflite'

# Convert the TensorFlow model to TFLite with TF Select ops enabled
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path_directory)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"ONNX model has been successfully converted to TFLite and saved at {tflite_model_path}")

ONNX model has been successfully converted to TFLite and saved at ./tf_lite_model.tflite


# Optimized tf_lite version

In [20]:

import tensorflow as tf

tf_model_path = './temp/'
tflite_model_path = './tf_lite_model_optimized.tflite'
# Convert the TensorFlow model to TFLite with TF Select ops enabled
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]

# Apply optimization for quantization
"there are different optimizations available, we're just using the default optimization"
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"ONNX model has been successfully converted to TFLite and saved at {tflite_model_path}")

ONNX model has been successfully converted to TFLite and saved at ./tf_lite_model_optimized.tflite


# Inference for quantized model

In [24]:
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="tf_lite_model_optimized.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'serving_default_images:0', 'index': 0, 'shape': array([  1,   3, 288, 288]), 'shape_signature': array([  1,   3, 288, 288]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 1223, 'shape': array([   1,   13, 1701]), 'shape_signature': array([   1,   13, 1701]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [25]:
# Prepare input data
input_data = np.random.randn(1,3,288,288).astype(np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process output data
print(output_data)


[[[     3.6731      18.563      22.119 ...       180.3      240.39      276.63]
  [     7.5418      5.4746      7.5929 ...      263.46      260.07       261.6]
  [     7.2051      33.413      38.729 ...      335.43      325.01      321.88]
  ...
  [ 6.7341e-06  1.6802e-06  3.8269e-07 ...  1.3927e-06    1.13e-06    1.01e-06]
  [ 1.6404e-08  5.7363e-10   1.573e-10 ...  9.4366e-07  8.5718e-07  8.0111e-07]
  [ 9.3082e-09  2.6777e-11  7.7509e-13 ...  1.0211e-06  9.2971e-07  8.7273e-07]]]


In [28]:
print(output_data.shape)

(1, 13, 1701)
